In [1]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import torchvision.transforms as T
import matplotlib.patches as patches

In [2]:
class CustomVOCImageDataset(Dataset):
    def __init__(self, annotations_dir, img_dir, transform=None, target_transform=None):
        self.annotations_dir = annotations_dir  # Thư mục chứa các file XML
        self.img_dir = img_dir  # Thư mục chứa các file ảnh
        self.transform = transform  # Biến đổi ảnh (nếu có)
        self.target_transform = target_transform  # Biến đổi nhãn (nếu có)
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]  # Lấy danh sách các file .jpg

    def __len__(self):
        return len(self.image_files)

    def parse_voc_xml(self, xml_file):
        """Hàm phân tích cú pháp file XML Pascal VOC"""
        tree = ET.parse(xml_file)
        root = tree.getroot()

        objects = []
        for obj in root.findall('object'):
            obj_name = obj.find('name').text
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            objects.append({
                'name': obj_name,
                'bbox': [xmin, ymin, xmax, ymax]
            })
        return objects

    def __getitem__(self, idx):
        # Lấy đường dẫn tới ảnh và đọc ảnh
        img_filename = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_filename)
        image = read_image(img_path)

        # Lấy đường dẫn tới file XML tương ứng
        xml_filename = img_filename.replace('.jpg', '.xml')
        xml_path = os.path.join(self.annotations_dir, xml_filename)

        # Đọc file XML để lấy nhãn và bounding box
        labels = self.parse_voc_xml(xml_path)

        # Biến đổi ảnh nếu cần
        if self.transform:
            image = self.transform(image)

        # Biến đổi nhãn nếu cần
        if self.target_transform:
            labels = self.target_transform(labels)

        return image, labels

In [3]:
# Khởi tạo dataset
annotations_dir = '/mnt/data/vietnam_traffic_sign_dataset/train'  # Thư mục chứa file XML
img_dir = '/mnt/data/vietnam_traffic_sign_dataset/train'  # Thư mục chứa file ảnh

# Tạo tập dữ liệu với transform cơ bản (ví dụ chuyển đổi sang tensor)
dataset = CustomVOCImageDataset(annotations_dir, img_dir, transform=None)

# Hiển thị một ảnh và bounding boxes từ dataset
def plot_image_with_boxes(image, labels):
    fig, ax = plt.subplots(1)
    ax.imshow(image.permute(1, 2, 0))  # Đổi trục từ (C, H, W) sang (H, W, C)

    # Vẽ bounding box cho từng object
    for label in labels:
        bbox = label['bbox']
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1], linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(bbox[0], bbox[1] - 5, label['name'], color='red', fontsize=12)

    plt.show()

# Lấy một ảnh và nhãn
image, labels = dataset[0]

# Hiển thị ảnh và bounding boxes
plot_image_with_boxes(image, labels)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/mnt/data/vietnam_traffic_sign_dataset/train'